
# Perceptrón Simple

In [ ]:
import sys
import numpy as np

sys.path.append('../scripts')

from utils import linear_sep_data
from perceptron import Perceptron

In [ ]:
n = 750
p = 7

x, y = linear_sep_data(n, p)

random_seeds = [1, 42, 42, 100, 2024]

for seed in random_seeds:
    np.random.seed(seed)
    perceptron = Perceptron(input_size = p, learning_rate = 0.1)
    print(f"Seed: {seed}, convergence epoch: {perceptron.train(x, y, epochs = 1000, verbose =False)}")